<img src='../../img/anaconda-logo.png' align='left' style="padding:10px">
<br>
*Copyright Continuum 2012-2016 All Rights Reserved.*

# Bokeh: Building Figures

We'll start by describing the available style properties that can be passed as kewyord arguments when the Figure object is created or by directly setting member attributes.

A more complete description of all styling options is available in the [Bokeh User Guide](#http://bokeh.pydata.org/en/latest/docs/user_guide/styling.html).

## Table of Contents
* [Building Figures](#Building-Figures)
	* [Basic Properties](#Basic-Properties)
	* [Dimensions](#Dimensions)
	* [Background](#Background)
	* [Title Text](#Title-Text)
	* [Border](#Border)
	* [Outline](#Outline)
	* [Axes](#Axes)
		* [Types](#Types)
		* [Ranges](#Ranges)
		* [Categorical Ranges](#Categorical-Ranges)
		* [Twin Axes](#Twin-Axes)
		* [Grid lines and Bands](#Grid-lines-and-Bands)
		* [Labels and Properties](#Labels-and-Properties)
		* [Tick Label Orientation](#Tick-Label-Orientation)
	* [Legends](#Legends)
* [Selecting objects](#Selecting-objects)
* [Exercise](#Exercise)


## Set-Up

In [1]:
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

In [2]:
# The best source for ALOT of information is bokeh.plotting
import bokeh.plotting
help(bokeh.plotting)

Help on package bokeh.plotting in bokeh:

NAME
    bokeh.plotting - # extra imports -- just things to add to 'from bokeh.plotting import'

PACKAGE CONTENTS
    figure
    helpers
    tests (package)

DATA
    DEFAULT_TOOLS = 'pan,wheel_zoom,box_zoom,save,reset,help'

FILE
    c:\anaconda3\envs\iqt\lib\site-packages\bokeh\plotting\__init__.py




To begin with the Figure class has many default properties like axes, ranges and tools to simplify generation of a plot using this interface.

## Basic Properties

The two basic properties can be applied to many of the elements of a figure are `text` and `line`.

For any text-based property such as `axis_label` or `title_text` the following properties can be adjusted by appending the style to the text property.
* `_text_font`
* `_text_font_size`
* `_text_font_style`
* `_text_color`
* `_text_alpha`
* `_text_baseline`

For any line-based property such as the `line` glyph, axis and grid lines the following properties can be adjusted by append the style to the line property
* `_line_width`
* `_line_color`
* `_line_alpha`
* `_line_dash`

In [4]:
import numpy as np

x = np.linspace(0, 4*np.pi, 100)
y = np.sin(x)

## Figure Dimensions

In [5]:
from bokeh.plotting import figure

plot = figure(plot_height=200)
plot.plot_width=1000
plot.line(x,y,line_width=2)
show(plot)

New in Bokeh 0.10 is the ability to make a `responsive` plot:
* ...that will resize based on the browser. 
* Try changing the the width and height of the plot below. 
* The plot will always be drawn a the full width of the notebook 
* ...but the ratio between width and height will be respected.

In [6]:
plot = figure(responsive=True)
plot.plot_width=1000
plot.plot_height=200
plot.line(x,y,line_width=2)
show(plot)

## Figure Background

In [7]:
plot = figure()
plot.background_fill_color='azure'
plot.line(x,y,line_width=2)
show(plot)

## Title Text

In [8]:
plot = figure()

plot.title.text_font_style='italic'
plot.title.text_font_size='50pt'
plot.title.text_color='olive'
plot.title.text = 'Sine Function'
plot.line(x,y,line_width=2)
show(plot)

## Border

In [9]:
plot = figure()
plot.border_fill_color='darkgray'
plot.min_border_bottom=20
plot.min_border_left=100
plot.min_border_top=0
plot.min_border_right=0
plot.line(x,y,line_width=2)
show(plot)

## Outline

In [10]:
plot = figure(outline_line_width=6,
             outline_line_alpha=0.3,
             outline_line_color='firebrick')
plot.line(x,y,line_width=2)
show(plot)

## Axes

Plot axes are separate objects that can be associated with one or more figures. The `figure` method automatically creates `xaxis` and `yaxis` objects.

### Types

The axis type can be changed using `x_axis_type` and `y_axis_type` keyword arguments to `figure()`.
* `linear` (default)
* `log`
* `datetime`

In [11]:
logX = 10**x

plot = figure(y_axis_type='log')
plot.line(x,logX,line_width=2)
show(plot)

The axes can also be modified by accessing the individual classes. `plot.yaxis` (and `plot.xaxis`) returns a *splattable* list of objects defined on the xaxis. Notice that LogAxis is a subclass of `axes` and the object would have to be replaced to change the axis to a LinearAxis. It is best to decide the axis type when the figure object is created.

In [ ]:
plot.xaxis,plot.yaxis

### Ranges

Ranges can be set in the figure using the `x_range` and `y_range` properties where the start and end points are given in a tuple.

By default `x_range` and `y_range` are `DataRange1d` objects that auto-fit the view to the input data.

We're going to replace `y_range` with a `Range1d` object, which takes start and end points as arguments to set the default view. As we'll see later it is useful to know that we can control the `x_range` and `y_range` objects. 

In [12]:
from bokeh.models import Range1d

logX = 10**x

plot = figure(y_axis_type='log',  x_range=(-10,20))
plot.y_range=Range1d(1,1000)
plot.line(x,logX,line_width=2)
show(plot)

### Categorical Ranges

By providing a list of 'factors' to the `y_range` parameter the yxais tic labels can easily be changed to arbitrary strings.

When using Pandas DataFrames this process can be simplified. (see below)

In [13]:
x2 = [50, 40, 65, 10, 25, 37, 80, 60]
factors = ["a", "b", "c", "d", "e", "f", "g", "h"]

plot = figure(y_range=factors)

plot.circle(x2, factors, size=15, fill_color="orange", line_color="green", line_width=3)
show(plot)

In [14]:
plot.y_range

### Twin Axes

Extra axes can be added as dictionary associations using the `extra_y_ranges` and `extra_x_ranges` member functions of the Figure object. The range is set in the dictionary using `Range1d`. The dictionary keys are arbitrary associations.

The new axis is rendered using the `add_layout` function.

In [15]:
from bokeh.models import LinearAxis, Range1d

plot = figure( y_range=(-1.2, 1.2) )
plot.line(x,y,line_width=2)

y2 = x**2

plot.extra_y_ranges = {"xSqRange": Range1d(start=0, end=100)}
plot.circle(x,y2, y_range_name='xSqRange', color='red')
plot.add_layout(LinearAxis(y_range_name="xSqRange"), 'left')

show(plot)

### Grid lines and Bands

Like axes grid lines are also controlled by `grid`, `xgrid` and `ygrid` members of Figure. Again they return a *splattable* list.

All of the [text properties](#Basic-Properties) apply. To operate on minor grid lines prepend `minor_` to the property attributes.

In [16]:
plot = figure( y_range=(-1.2, 1.2) )
plot.line(x,y,line_width=2)

plot.grid.grid_line_width=1.5

plot.ygrid.grid_line_color='blue'

#remove grid lines intersecting the x axis
plot.xgrid.grid_line_color = None

plot.ygrid.minor_grid_line_color='gray'
plot.ygrid.minor_grid_line_width=1

show(plot)

In [17]:
plot = figure( y_range=(-1.2, 1.2) )
plot.line(x,y,line_width=2)

#remove all grid lines
plot.grid.grid_line_color = None

plot.ygrid.band_fill_alpha = 0.1
plot.ygrid.band_fill_color = "navy"

show(plot)

### Labels and Properties

Axis objects are available as through the `xaxis`, `yaxis` and `axis` member objects of a Figure. Each of these return a *splattable* list of axis objects.

The line properties can be changed with
* `axis_line_width`
* `axis_line_color`
* `axis_line_alpha`

and similiarly for text properties.

In [18]:
plot = figure( y_range=(-1.2, 1.2) )
plot.line(x,y,line_width=2)

plot.extra_y_ranges = {"linear": Range1d(start=0, end=100)}
plot.circle(x,y2, y_range_name='linear', color='red')
plot.add_layout(LinearAxis(y_range_name="linear"), 'left')

#use bold fonts for all axis labels
plot.axis.axis_label_text_font_style='bold'

#label the x axis
plot.xaxis.axis_label='x'
plot.xaxis.axis_line_width=4

#the first y axis
plot.yaxis[0].axis_label='sin(x)'
plot.yaxis[0].axis_label_text_color='blue'

#the second yaxis
plot.yaxis[1].axis_label='x^2'
plot.yaxis[1].axis_label_text_color='red'

show(plot)

### Tick Label Orientation

The orientation of the major labels can be changed with `major_label_orentation`. The possible values are 
* `horizontal`
* `vertical`
* Degrees in radians
    

In [23]:
from bokeh.sampledata.glucose import data as glucose
import numpy as np

#reduce data size
glucose = glucose.ix['2010-10-06':'2010-10-13']

plot = figure(x_axis_type="datetime")
plot.line(glucose.index.to_series(), glucose['glucose'], line_color='gray')

plot.xaxis.major_label_orientation=np.pi/4

show(plot)

## Legends

Legends are automatically added to figures when the `legend` parameter is passed to a glyph. By default the legend appears at the top-right of the Figure.

The `legend` member object (*splattable* list) is used to set legend position and properties.

* What happens when the order of the calls to glyph methods is changed?

In [20]:
plot = figure()

plot.line(x,y, line_color='blue', legend='sin(x)')
plot.circle(x,y, line_color='blue', fill_color='blue',size=5, legend='sin(x)')

plot.line(x,2*y,line_dash=[4,4], line_color='darkorchid', legend='2sin(x)')

plot.line(x,3*y, line_color='green', legend='3sin(x)')
plot.circle(x,3*y, line_color='green', fill_color='white', legend='3sin(x)')

plot.legend.location='bottom_left'
plot.legend.border_line_width = 3

show(plot)

# Selecting objects

Since everything in a Bokeh plot is an object a special `select()` method has been provided to switch between objects within a plot to apply styling properties.

Many methods in the `plotting` interface take the `name` paramter that is very useful when using the `select` tool.

NOTE: Not all glyphs have the same parameters.



In [21]:
plot = figure()

plot.line(x,y, name='SineLine')
plot.circle(x,y, name='SineCircle')

plot.line(x,2*y, name='2SineLine')

plot.line(x,3*y, name='3SineLine')
plot.circle(x,3*y, name='3SineCircle')

for m_name in ['SineLine', 'SineCircle']:
    obj = plot.select(name=m_name)[0].glyph
    try:
        obj.fill_color='green'
    except AttributeError:
        #line objects don't have fill colors
        obj.line_color='green'


show(plot)

The class itself can be used to return a list of all objects of that type

# Exercise

<img src='img/topics/Exercise.png' align='left' style='padding:10px'>
<br>
<a href='./Bokeh_ex_plotting.ipynb' class='btn btn-primary btn-lg'>Plotting Timeseries</a>

----
<a href='./Bokeh_layout.ipynb' class='btn btn-primary'>Plot layout</a>